In [38]:
import utils
import torch
from model_search import Network
import torchvision.datasets as dset
from torch.autograd import Variable

def infer(valid_queue, model, criterion):
  objs = utils.AvgrageMeter()
  top1 = utils.AvgrageMeter()
  top5 = utils.AvgrageMeter()
  model.eval()

  for step, (input, target) in enumerate(valid_queue):
    input = Variable(input, volatile=True).cuda()
    target = Variable(target, volatile=True).cuda()

    logits = model(input)
    loss = criterion(logits, target)

    prec1, prec5 = utils.accuracy(logits, target, topk=(1, 5))
    n = input.size(0)
    objs.update(loss.data[0], n)
    top1.update(prec1.data[0], n)
    top5.update(prec5.data[0], n)

    if step % args.report_freq == 0:
        print('valid',step, objs.avg, top1.avg, top5.avg)

  return top1.avg, objs.avg

from collections import namedtuple
args = namedtuple("args", ['cutout', 'report_freq'])(False, 10)
train_transform, valid_transform = utils._data_transforms_cifar10(args)
train_data = dset.CIFAR10('/home/yuezx/dataset.yzx/', train=True, download=True, transform=train_transform)

Files already downloaded and verified


In [41]:
criterion = torch.nn.CrossEntropyLoss()
criterion = criterion.cuda()

model = Network(16, 10, 8, criterion)
model.load_state_dict(torch.load("/home/yuezx/darts/cnn/search-EXP-20200902-231525/weights.pt"))
model = model.cuda()

valid_queue = torch.utils.data.DataLoader(
      train_data, batch_size=10,
      sampler=torch.utils.data.sampler.SubsetRandomSampler(range(10000)[:1000]),
      pin_memory=True, num_workers=2)

valid_acc, valid_obj = infer(valid_queue, model, criterion)

valid 0 8.773923873901367 0.0 0.0
valid 10 5.835864760658958 7.2727272727272725 35.45454545454545
valid 20 5.343898023877825 8.571428571428571 47.142857142857146
valid 30 5.27610795728622 8.709677419354838 48.064516129032256
valid 40 5.124170820887496 10.0 50.48780487804878
valid 50 5.072431727951648 10.392156862745098 51.372549019607845
valid 60 5.046553857990952 10.655737704918034 51.14754098360656
valid 70 5.068150456522552 10.0 50.985915492957744
valid 80 5.009497739650585 10.37037037037037 51.72839506172839
valid 90 5.014650819065807 10.32967032967033 51.20879120879121


In [40]:
valid_acc, valid_obj

(10.8, 101740.5732421875)